In [1]:
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
# !pip install impyute
# !pip install fancyimpute
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as skLDA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy import stats
import numpy as np
# import impyute as impy
from fancyimpute import SoftImpute#, MatrixFactorization
import pandas as pd
import time
import math
from missingpy import MissForest
from sklearn.model_selection import train_test_split
from numpy.linalg import norm, inv

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from fancyimpute import SimpleFill, SoftImpute
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import InstanceHardnessThreshold

from imblearn.metrics import geometric_mean_score
from imblearn.metrics import sensitivity_score

from sklearn.metrics import f1_score, roc_auc_score
from sklearn import metrics
from imblearn.datasets import fetch_datasets
from imblearn.over_sampling import KMeansSMOTE
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SVMSMOTE
from functions.OsImp import *
import warnings
warnings.filterwarnings("ignore")

from functions.eval import *
from functions.utils import *
from functions.OsImp import *
from functions.OsImp_star import *
from functions.dpers import *
from functions.dimv import dimv
# imputer = dimv

imputer = IterativeImputer().fit_transform
# imputer = SoftImpute(convergence_threshold=0.1, max_iters=10).fit_transform
# imputer = MissForest().fit_transform

In [2]:
data = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/parkinsons/parkinsons.data',
                  sep = ",")
data = data.drop(['name'], axis = 1)
X, y = data.drop(['status'], axis = 1), data['status']
G = 2
print(np.array([sum(y==0), sum(y==1)]))
print(X.shape)
X.head()

[ 48 147]
(195, 22)


,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),...,MDVP:APQ,Shimmer:DDA,NHR,HNR,RPDE,DFA,spread1,spread2,D2,PPE
0,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,0.426,...,0.02971,0.06545,0.02211,21.033,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,0.626,...,0.04368,0.09403,0.01929,19.085,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,0.482,...,0.03590,0.08270,0.01309,20.651,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,0.517,...,0.03772,0.08771,0.01353,20.644,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,0.584,...,0.04465,0.10470,0.01767,19.649,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [3]:
X = X.to_numpy()
y = np.asarray(y)

In [4]:
missing_rates = np.array([0.5, 0.6, 0.7, 0.8, 0.9])
res_all = []
for missing_rate in missing_rates:
  K, nMC, result = 2, 10, []
  n = np.array([100,-80])
  res = np.asarray([get_all_res_star(X, y, i, n, K, G,imputer, missing_rate) for i in range(nMC)], dtype = object)
  res = show_result(res)
  res_all.append(res)    

In [5]:
concat_tab = pd.concat((res_all[0][1][0], res_all[0][1][1], 
                        res_all[1][1][0], res_all[1][1][1], 
                        res_all[2][1][0], res_all[2][1][1], 
                        res_all[3][1][0], res_all[3][1][1], 
                        res_all[4][1][0], res_all[4][1][1]))

In [6]:
leftindex = pd.MultiIndex.from_product([["50%", "60%", "70%", "80%", "90%"],
                                        ["F-1 score","sensitivity"], 
                                        ["SVM", "LR", "DCT"]], 
                                       names = ["missing rate", "metric", "classifier"])
all_res = pd.DataFrame(concat_tab.to_numpy(), index = leftindex, columns = concat_tab.columns)
all_res

NearMiss         kmeanSmt  \
missing rate metric      classifier                                     
50%          F-1 score   SVM          0.596$\pm$0.08  0.763$\pm$0.023   
                         LR          0.758$\pm$0.047  0.759$\pm$0.021   
                         DCT         0.652$\pm$0.104   0.79$\pm$0.053   
             sensitivity SVM         0.568$\pm$0.077  0.744$\pm$0.026   
                         LR          0.745$\pm$0.057   0.74$\pm$0.024   
                         DCT         0.627$\pm$0.114  0.783$\pm$0.061   
60%          F-1 score   SVM         0.576$\pm$0.066  0.752$\pm$0.038   
                         LR          0.743$\pm$0.059   0.743$\pm$0.04   
                         DCT         0.537$\pm$0.171  0.774$\pm$0.031   
             sensitivity SVM          0.549$\pm$0.07  0.737$\pm$0.043   
                         LR          0.733$\pm$0.065  0.728$\pm$0.045   
                         DCT         0.528$\pm$0.159  0.773$\pm$0.033   
70%          F-1 score   SVM         0.508$\pm$0.117   0.755$\pm$0.02   
                         LR          0.746$\pm$0.068   0.754$\pm$0.04   
                         DCT          0.54$\pm$0.163  0.739$\pm$0.043   
             sensitivity SVM         0.494$\pm$0.114  0.742$\pm$0.027   
                         LR          0.733$\pm$0.071   0.74$\pm$0.047   
                         DCT         0.535$\pm$0.159  0.744$\pm$0.047   
80%          F-1 score   SVM         0.373$\pm$0.085   0.72$\pm$0.041   
                         LR          0.746$\pm$0.047  0.705$\pm$0.044   
                         DCT          0.45$\pm$0.135  0.726$\pm$0.076   
             sensitivity SVM         0.388$\pm$0.068  0.696$\pm$0.044   
                         LR          0.723$\pm$0.052  0.678$\pm$0.045   
                         DCT         0.433$\pm$0.128  0.729$\pm$0.071   
90%          F-1 score   SVM         0.254$\pm$0.095  0.681$\pm$0.075   
                         LR          0.676$\pm$0.063  0.664$\pm$0.084   
                         DCT         0.388$\pm$0.136  0.633$\pm$0.104   
             sensitivity SVM         0.333$\pm$0.053  0.682$\pm$0.066   
                         LR          0.656$\pm$0.063  0.644$\pm$0.085   
                         DCT         0.404$\pm$0.108   0.64$\pm$0.114   

                                               SmtNN           SVMSmt  \
missing rate metric      classifier                                     
50%          F-1 score   SVM         0.722$\pm$0.027   0.756$\pm$0.03   
                         LR          0.725$\pm$0.037  0.758$\pm$0.028   
                         DCT         0.748$\pm$0.055  0.799$\pm$0.041   
             sensitivity SVM         0.699$\pm$0.024  0.736$\pm$0.033   
                         LR           0.703$\pm$0.04   0.74$\pm$0.028   
                         DCT         0.731$\pm$0.055   0.79$\pm$0.046   
60%          F-1 score   SVM         0.716$\pm$0.045   0.752$\pm$0.04   
                         LR          0.722$\pm$0.042  0.761$\pm$0.052   
                         DCT         0.718$\pm$0.065  0.768$\pm$0.041   
             sensitivity SVM         0.696$\pm$0.051  0.737$\pm$0.048   
                         LR          0.703$\pm$0.049  0.747$\pm$0.059   
                         DCT         0.699$\pm$0.074   0.76$\pm$0.045   
70%          F-1 score   SVM         0.721$\pm$0.044  0.755$\pm$0.041   
                         LR          0.704$\pm$0.046  0.764$\pm$0.042   
                         DCT         0.744$\pm$0.038  0.705$\pm$0.067   
             sensitivity SVM         0.705$\pm$0.044  0.745$\pm$0.039   
                         LR          0.688$\pm$0.046  0.751$\pm$0.046   
                         DCT         0.732$\pm$0.039  0.699$\pm$0.063   
80%          F-1 score   SVM         0.671$\pm$0.068  0.717$\pm$0.043   
                         LR          0.689$\pm$0.049  0.717$\pm$0.051   
                         DCT           0.7$\pm$0.098  0.692$\pm$0.054   
             sensitivity SVM

In [7]:
concat_to_export = pd.concat((res_all[0][0][0][0], res_all[0][0][0][1], 
                        res_all[1][0][0][0], res_all[1][0][0][1], 
                        res_all[2][0][0][0], res_all[2][0][0][1], 
                        res_all[3][0][0][0], res_all[3][0][0][1], 
                        res_all[4][0][0][0], res_all[4][0][0][1]))
concat_to_export.to_csv("mean_f1_sensi_parkinson-high.csv", index = False)

In [8]:
print(all_res.to_latex(index = True, 
                       formatters = {"name": str.upper},
                      float_format = "{.1f}".format))

\begin{tabular}{lllllllll}
\toprule
    &             &     &         NearMiss &         kmeanSmt &            SmtNN &           SVMSmt &          InsHard &            OsImp \\
missing rate & metric & classifier &                  &                  &                  &                  &                  &                  \\
\midrule
50\% & F-1 score & SVM &   0.596\$\textbackslash pm\$0.08 &  0.763\$\textbackslash pm\$0.023 &  0.722\$\textbackslash pm\$0.027 &   0.756\$\textbackslash pm\$0.03 &   0.703\$\textbackslash pm\$0.03 &  0.755\$\textbackslash pm\$0.032 \\
    &             & LR &  0.758\$\textbackslash pm\$0.047 &  0.759\$\textbackslash pm\$0.021 &  0.725\$\textbackslash pm\$0.037 &  0.758\$\textbackslash pm\$0.028 &  0.694\$\textbackslash pm\$0.039 &  0.762\$\textbackslash pm\$0.028 \\
    &             & DCT &  0.652\$\textbackslash pm\$0.104 &   0.79\$\textbackslash pm\$0.053 &  0.748\$\textbackslash pm\$0.055 &  0.799\$\textbackslash pm\$0.041 &  0.656\$\textbackslash p